In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import pickle
sns.set_theme(style="darkgrid")

#Word2Vec
from langdetect import detect
import nltk
from nltk.tokenize import word_tokenize
import gensim
from gensim.models import Word2Vec

#keras
import tensorflow as tf
import keras

#Navneet
import string
import re
from tqdm import tqdm
import pandas as pd
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize
import preprocessor as p
from nltk.corpus import stopwords
import emoji
from sklearn.metrics import *

Using TensorFlow backend.


In [2]:
def func_Eval(Y_Actual,Y_pred):

    TrueNegative=0
    TruePositive=0
    FalsePositive=0
    FalseNegative=0
    for i in range(len(Y_pred)):
        #if the actual class is negative
        if(Y_Actual[i]==0):
            
            #if the predicited class is negative
            if(Y_pred[i]==0):
                TrueNegative+=1
                
            #if the predicted class is positive
            else:
                FalsePositive+=1
                
        #if the actual class is positive
        else:
            #if the predicited class is positive
            if(Y_pred[i]==1):
                TruePositive+=1
                
            #if the predicited class is negative
            else:
                FalseNegative+=1

    Confusion_Matrix=[[TrueNegative,FalsePositive],[FalseNegative,TruePositive]]
    
    Confusion_Matrix=pd.DataFrame(Confusion_Matrix,columns=['Predicted Negative','Predicted Positive'])
    
    Confusion_Matrix.rename(index={0: "Actual Negative", 1: "Actual Positive"},inplace=True)
    
    MyPrecision=TruePositive/(TruePositive+FalsePositive)

    MyRecall=TruePositive/(TruePositive+FalseNegative)
    
    MyAccuracy=(TruePositive+TrueNegative)/(TruePositive+TrueNegative+FalseNegative+FalsePositive)
    
    MyF1score= 2*(MyPrecision*MyRecall)/(MyPrecision+MyRecall)
    
    MyPrecisionZero=TrueNegative/(TrueNegative+FalseNegative)

    MyRecallZero=TrueNegative/(TrueNegative+FalsePositive)
    
    
    MyF1scoreZero= 2*(MyPrecisionZero*MyRecallZero)/(MyPrecisionZero+MyRecallZero)

    
    print("-----------------")
    print("Accuracy:",MyAccuracy)
    print("-----------------")

    print()

    print("For the Positive CLass, Label:4")
    print("-----------------")
    print("Precision:",MyPrecision)

    print("-----------------")
    print("Recall:",MyRecall)

    print("-----------------")
    print("F1 Score:",MyF1score)

    print()

    print("For the Negative CLass, Label:0")

    print("-----------------")
    print("Precision:",MyPrecisionZero)

    print("-----------------")
    print("Recall:",MyRecallZero)

    print("-----------------")
    print("F1 Score:",MyF1scoreZero)

    print()
    print("--------")
    print("Macro Average F1",(MyF1score+MyF1scoreZero)/2)
    print("-----------------")
    print("Confusion Matrix:")
    print(Confusion_Matrix)

In [3]:
def getF1(confusion):
    print("Accuracy:", getAccuracy(confusion))
    numClass = confusion.shape[0]
    total = np.sum(np.sum(confusion))
    
    # get True positive from diagonals
    tp = confusion.diagonal().reshape((numClass,1)).copy()
    
    # make diagonal zeros
    confusionCopy = confusion.copy()
    np.fill_diagonal(confusionCopy,0)
    fn = np.sum(confusionCopy,axis=1).reshape(((numClass,1))).copy()
    fp = np.sum(confusionCopy,axis=0).reshape(((numClass,1))).copy()
    if(np.sum(tp+fp)==0 or np.isfinite(tp+fp).all()==False):
        print(tp,fp)
        prec = tp
    else:
        prec = tp / (tp+fp)
    recall = tp / (tp+fn)
    f1 = (2*prec*recall) / (prec+recall)
    
    microPrec = np.sum(tp) / (np.sum(tp)+np.sum(fp))
    microRecall = np.sum(tp) / (np.sum(tp)+np.sum(fn))
    microf1 = (2*microPrec*microRecall) / (microPrec+microRecall)
    
    macroPrec = np.mean(prec)
    macroRecall = np.mean(recall)
    macrof1 = np.mean(f1)
    
    return [[microPrec,microRecall,microf1],[macroPrec,macroRecall,macrof1]]

def getConfusion(yTrue,yPred,numClass,cutoff = 0.5):
    yPred = np.where(yPred > cutoff, 1, 0)
    n=yTrue.shape[0]
    assert(yTrue.shape[0]==yPred.shape[0])
    confusion = np.zeros((numClass,numClass)).astype(int)
    for i in range(n):
        confusion[int(yTrue.item(i)),int(yPred.item(i))] +=1
    return confusion
def getAccuracy(confusion):
    total = np.sum(np.sum(confusion))
    tp = np.sum(confusion.diagonal())
    return (tp/total) * 100

In [4]:
def utils_preprocess_text(text, flg_stemm=False, flg_lemm=True, lst_stopwords=None):
    ## clean (convert to lowercase and remove punctuations and characters and then strip)
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'19', 'Z', str(text).lower().strip())
    text = re.sub(r'[0-9]+k*', ' X ', str(text).lower().strip())
    ## Tokenize (convert from string to list)
    lst_text = text.split()    ## remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in lst_stopwords]

    ## Stemming (remove -ing, -ly, ...)
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]

    ## Lemmatisation (convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]

    ## back to string from list
    text = " ".join(lst_text)
    return text
def processdf(df):
    if('label' in df.columns):
        df['label'] = np.where(df['label']=="fake",0,1)
    else:
        df['label'] = np.zeros(len(df)).astype(np.int)
    df['tweet'] = df['tweet'].apply(utils_preprocess_text)
    df['tweet'] = df['tweet'].apply(word_tokenize)

In [5]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textstat import flesch_reading_ease,flesch_kincaid_grade
def getLexicon1():
    d_emotion=pd.read_csv("./lexicons/5. NRC-Hashtag-Emotion-Lexicon-v0.2.txt", sep='\t', names=['emotion','word','score'], header=None)
    dic_dEmo={}
    for i in range(len(d_emotion['word'])):
        if(type(d_emotion['word'][i]) == str):
            turncated = d_emotion['word'][i]
            if(turncated[0]=='#'):
                turncated = turncated[1:]
            dic_dEmo[turncated]=d_emotion['score'][i]
    return dic_dEmo
def getLexicon2():
    d_emotion=pd.read_csv("./lexicons/6. NRC-10-expanded.csv", sep='\t')
    dic_dEmo={}
    for i in range(len(d_emotion['word'])):
        if(type(d_emotion['word'][i]) == str):
            turncated = d_emotion['word'][i]
            if(turncated[0]=='#'):
                turncated = turncated[1:]
            dic_dEmo[turncated]=d_emotion['anger'][i]
    return dic_dEmo
def getLexicon3():
    d_emotion=pd.read_csv("./lexicons/8. NRC-word-emotion-lexicon.txt", sep='\t', names=['word','emotion','score'], header=None)
    dic_dEmo={}
    for i in range(len(d_emotion['word'])):
        if(type(d_emotion['word'][i]) == str):
            turncated = d_emotion['word'][i]
            if(turncated[0]=='#'):
                turncated = turncated[1:]
            dic_dEmo[turncated]=d_emotion['score'][i]
    return dic_dEmo
def vader(GivenTweets):
    obj= SentimentIntensityAnalyzer()
    Comp_vader=np.zeros(len(GivenTweets))
    for i in range(len(GivenTweets)):
        sentiment_dict = obj.polarity_scores(GivenTweets[i])
        Comp_vader[i]=sentiment_dict['compound']
    return(Comp_vader)
def reading_ease(GivenTweets):
    Reading_Ease=np.zeros(len(GivenTweets))
    Reading_Grade=np.zeros(len(GivenTweets))
    for i in range(len(GivenTweets)):
        joinedString = "".join(GivenTweets[i])
#         print(GivenTweets[i])
        Reading_Ease[i]=flesch_reading_ease(joinedString)
        Reading_Grade[i]=flesch_kincaid_grade(joinedString)
    return(Reading_Ease,Reading_Grade)
def slang(GivenTweets):
    slangs= pd.read_csv("./lexicons/SlangSD/SlangSD.txt", sep='\t', names=['word','score'], header=None)
    slangScore=np.zeros(len(GivenTweets))
    dic_slangs={}
    for i in (range(len(slangs['word']))):
        dic_slangs[slangs['word'][i]]=slangs['score'][i]
    # print(dic_slangs)
    for i in (range(len(GivenTweets))):
        tweet=GivenTweets[i]
        for j in range(len(tweet)):
            if(tweet[j] in dic_slangs):
    # #             print(i,dic_emoji[i])
                slangScore[i]+=dic_slangs[tweet[j]]
    return(slangScore)
def getLength(GivenTweets):
    ret=np.zeros(len(GivenTweets))
    for i in (range(len(GivenTweets))):
        tweet=GivenTweets[i]
        ret[i] = len(tweet)
    return ret
def getAvgWord(GivenTweets):
    ret=np.zeros(len(GivenTweets))
    for i in (range(len(GivenTweets))):
        tweet=GivenTweets[i]
        cnt = 0
        add = 0
        for j in range(len(tweet)):
            cnt+=1
            add+= len(tweet[j])
        ret[i] = add/cnt
    return ret
def getSpecial(GivenTweets):
    ret=np.zeros(len(GivenTweets))
    for i in (range(len(GivenTweets))):
        tweet=GivenTweets[i]
        cnt = 0
        for j in range(len(tweet)):
            if(tweet[j]=='X'):
                cnt+=1
        ret[i] = cnt
    return ret

In [6]:
from gensim.models import FastText
from sklearn.feature_extraction.text import CountVectorizer
def helpergun(x):
    return " ".join(x)
def saveWord2VecModel(corpusList):
    corpus = pd.concat([x['tweet'] for x in corpusList])
    # WORD2VEC
    vecmodel = gensim.models.word2vec.Word2Vec(corpus, size=VECTOR_SIZE,window=8, min_count=1, sg=1, iter=50)
    # FASTTEXT
    # vecmodel = FastText(corpus, size=VECTOR_SIZE, window=8, min_count=5, workers=4,sg=1)
    open("word2Vec",'wb').write(pickle.dumps(vecmodel))
    return vecmodel

VECTOR_SIZE = 200
EXTRA_VECTORS = 3
PADDING_SIZE = 40
SENT_LEVEL_FEATURES = 5
UNIGRAMS_CNT = 4070

def getEmbeddingsFeatures(corpusList,vecmodel):
    corpus = pd.concat([x['tweet'] for x in corpusList])
    
    corpusJoined = corpus.apply(helpergun)
    
    vectorizer = CountVectorizer(ngram_range=(1,1),min_df=5)
    unigrams =vectorizer.fit_transform(corpusJoined)
    
    X_unig = []
    for df in corpusList:
        X_unig.append(vectorizer.transform(df['tweet'].apply(helpergun)))
    
    # Index words with corresponding index number and fit on corpus
    tk=tf.keras.preprocessing.text.Tokenizer(lower=True,split=' ',oov_token="NaN", filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
    tk.fit_on_texts(corpus)

    # Covert sentences to index sequences
    for df in corpusList:
        df['tweet_id'] = tk.texts_to_sequences(df['tweet'])

    # Pad sequences
    Xlist = [] 
    for df in corpusList:
        Xlist.append(tf.keras.preprocessing.sequence.pad_sequences(df['tweet_id'], maxlen=PADDING_SIZE,padding="post", truncating="post",value=0))

    ylist = []
    for df in corpusList:
        ylist.append(np.array(df['label']).reshape((len(df),1)))
        
    # EMBEDDING START
    # ====================================================================================
    # print(tk.word_index)
    # LEXICON FEATURES APPEND
    dic1 = getLexicon1()
    dic2 = getLexicon2()
    dic3 = getLexicon3()

    embed = np.zeros((len(tk.word_index)+1, VECTOR_SIZE+EXTRA_VECTORS))
    for word,idx in tk.word_index.items():
        if(word in vecmodel.wv):
    #         embed[idx] = vecmodel.wv[word]
            lexiconVec = np.zeros(EXTRA_VECTORS)
            if(word in dic1):
                lexiconVec[0] = dic1[word]
            if(word in dic2):
                lexiconVec[1] = dic2[word]
            if(word in dic3):
                lexiconVec[2] = dic3[word]
            embed[idx] = np.concatenate((vecmodel.wv[word],lexiconVec))
    # ====================================================================================
    # EMBEDDING END
    
    # SENT LEVEL FEATURES START
    XFlist = []
    gh = 0
    for df in corpusList:
        XF = np.zeros((len(df),SENT_LEVEL_FEATURES+UNIGRAMS_CNT))
#         XF[:,0] = vader(df['tweet'])
        re,rg = reading_ease(df['tweet'])
        XF[:,0] = re
        XF[:,1] = rg
#         XF[:,3] = slang(df['tweet'])
        XF[:,2] = getLength(df['tweet'])
        XF[:,3] = getAvgWord(df['tweet'])
        XF[:,4] = getSpecial(df['tweet'])
        XF[:,5:] = X_unig[gh].toarray()
        XFlist.append(XF)
        gh+=1
    
    for i in range(len(Xlist)):
        print(Xlist[i].shape,ylist[i].shape)
    
    return Xlist,ylist,XFlist,embed

In [62]:
file=open("train_feature","rb")
xt=pickle.load(file)
file.close()
file=open("train_label","rb")
yt=pickle.load(file)
file.close()
file=open("valid_label","rb")
yv=pickle.load(file)
file.close()
file=open("valid_feature","rb")
xv=pickle.load(file)
file.close()

file=open("test_feature","rb")
testX=pickle.load(file)
file.close()

In [43]:
yt = np.where(yt=="real",1,0)
yv = np.where(yv=="real",1,0)
yt.shape

(6420,)

In [44]:
xt = xt.tocsr()
xv = xv.tocsr()
testX = testX.tocsr()

In [51]:
y_pred

array([[0.4544735 ],
       [0.5367213 ],
       [0.4613737 ],
       ...,
       [0.52380615],
       [0.4835064 ],
       [0.6042369 ]], dtype=float32)

In [60]:
with tf.device('/CPU:0'):
    x_in = keras.layers.Input(shape=(xt.shape[1],))
    x = Dense(32, activation='relu')(x_in)
    x = BatchNormalization()(x)
    x = Dense(10, activation='relu')(x)     
    y_out = Dense(1, activation='sigmoid')(x)
    model = keras.models.Model(x_in,y_out)
    model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
    print(model.summary())
    training = model.fit(xt,yt,batch_size=64,epochs=10, shuffle="batch", verbose=1, validation_split=0)
    y_pred = model.predict(xv)
    y_cut = np.where(y_pred>0.5,1,0)
    macrof1 = f1_score(yv, y_cut, zero_division=1)
    print("MACRO F1 : ",macrof1)

Model: "model_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_20 (InputLayer)        (None, 28601)             0         
_________________________________________________________________
dense_45 (Dense)             (None, 32)                915264    
_________________________________________________________________
batch_normalization_1 (Batch (None, 32)                128       
_________________________________________________________________
dense_46 (Dense)             (None, 10)                330       
_________________________________________________________________
dense_47 (Dense)             (None, 1)                 11        
Total params: 915,733
Trainable params: 915,669
Non-trainable params: 64
_________________________________________________________________
None
Epoch 1/10
6420/6420 [==============================] - 4s 675us/step - loss: 0.2645 - accuracy: 0.9048
Epoch 2/

In [70]:
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.utils import shuffle
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import BaggingClassifier

from scipy.sparse import coo_matrix, vstack
X_comb = vstack([xt,xv])
y_comb = np.concatenate([np.array(yt),np.array(yv)])
y_comb = np.where(y_comb=="real",1,0)
X_comb = X_comb.tocsr()
testX = testX.tocsr()
X_comb, y_comb = shuffle(X_comb,y_comb)
Xsplits=[]
Ysplits=[]
size = X_comb.shape[0]
K=10
sizeOfsplit = size//K
for i in range(K):
    Xsplits.append(X_comb[i*sizeOfsplit:(i+1)*sizeOfsplit,:])
    Ysplits.append(y_comb[i*sizeOfsplit:(i+1)*sizeOfsplit])
splits = [ tuple(filter(lambda x: x!=i,tuple(range(K)))) for i in range(K)]
res = []
testRes=[]
for i in range(K):
    Xtrain = vstack(tuple(Xsplits[x] for x in splits[i]))
    ytrain = np.concatenate(tuple(Ysplits[x] for x in splits[i]))
    
    Xtest = Xsplits[i]
    ytest = np.copy(Ysplits[i])
    
    with tf.device('/CPU:0'):
        x_in = keras.layers.Input(shape=(xt.shape[1],))
        x = Dense(64, activation='relu')(x_in)
#         x = BatchNormalization()(x)
#         Dropout(0.2)
        x = Dense(16, activation='relu')(x)  
#         Dropout(0.2)
#         x = BatchNormalization()(x)
        y_out = Dense(1, activation='sigmoid')(x)
        model = keras.models.Model(x_in,y_out)
        model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
#         print(model.summary())
        training = model.fit(Xtrain,ytrain,batch_size=64,epochs=10, shuffle="batch", verbose=2, validation_split=0,validation_data=(Xtest, ytest))
        y_pred = model.predict(Xtest)
        y_cut = np.where(y_pred>0.5,1,0)
        macrof1 = f1_score(ytest, y_cut, zero_division=1)
        print("MACRO F1 : ",macrof1)

Train on 7704 samples, validate on 856 samples
Epoch 1/10
 - 8s - loss: 0.3913 - accuracy: 0.8830 - val_loss: 0.1217 - val_accuracy: 0.9766
Epoch 2/10
 - 8s - loss: 0.0483 - accuracy: 0.9910 - val_loss: 0.0643 - val_accuracy: 0.9766
Epoch 3/10
 - 8s - loss: 0.0094 - accuracy: 0.9994 - val_loss: 0.0567 - val_accuracy: 0.9778
Epoch 4/10
 - 8s - loss: 0.0034 - accuracy: 0.9999 - val_loss: 0.0527 - val_accuracy: 0.9825
Epoch 5/10
 - 8s - loss: 0.0017 - accuracy: 1.0000 - val_loss: 0.0523 - val_accuracy: 0.9813
Epoch 6/10
 - 8s - loss: 9.6766e-04 - accuracy: 1.0000 - val_loss: 0.0515 - val_accuracy: 0.9825
Epoch 7/10
 - 8s - loss: 6.1483e-04 - accuracy: 1.0000 - val_loss: 0.0508 - val_accuracy: 0.9825
Epoch 8/10
 - 8s - loss: 4.1654e-04 - accuracy: 1.0000 - val_loss: 0.0508 - val_accuracy: 0.9825
Epoch 9/10
 - 8s - loss: 2.9615e-04 - accuracy: 1.0000 - val_loss: 0.0506 - val_accuracy: 0.9848
Epoch 10/10
 - 8s - loss: 2.1833e-04 - accuracy: 1.0000 - val_loss: 0.0509 - val_accuracy: 0.9848
MA

Train on 7704 samples, validate on 856 samples
Epoch 1/10
 - 8s - loss: 0.3716 - accuracy: 0.9190 - val_loss: 0.1245 - val_accuracy: 0.9708
Epoch 2/10
 - 8s - loss: 0.0446 - accuracy: 0.9917 - val_loss: 0.0844 - val_accuracy: 0.9708
Epoch 3/10
 - 8s - loss: 0.0089 - accuracy: 0.9992 - val_loss: 0.0810 - val_accuracy: 0.9708
Epoch 4/10
 - 8s - loss: 0.0033 - accuracy: 1.0000 - val_loss: 0.0822 - val_accuracy: 0.9696
Epoch 5/10
 - 8s - loss: 0.0018 - accuracy: 1.0000 - val_loss: 0.0845 - val_accuracy: 0.9708
Epoch 6/10
 - 8s - loss: 0.0011 - accuracy: 1.0000 - val_loss: 0.0880 - val_accuracy: 0.9720
Epoch 7/10
 - 8s - loss: 7.4652e-04 - accuracy: 1.0000 - val_loss: 0.0894 - val_accuracy: 0.9708
Epoch 8/10
 - 8s - loss: 5.3934e-04 - accuracy: 1.0000 - val_loss: 0.0904 - val_accuracy: 0.9708
Epoch 9/10
 - 8s - loss: 4.0731e-04 - accuracy: 1.0000 - val_loss: 0.0934 - val_accuracy: 0.9708
Epoch 10/10
 - 8s - loss: 3.1729e-04 - accuracy: 1.0000 - val_loss: 0.0949 - val_accuracy: 0.9708
MACRO 

In [71]:
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.utils import shuffle
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import BaggingClassifier

from scipy.sparse import coo_matrix, vstack
X_comb = vstack([xt,xv])
y_comb = np.concatenate([np.array(yt),np.array(yv)])
y_comb = np.where(y_comb=="real",1,0)
X_comb = X_comb.tocsr()
testX = testX.tocsr()
X_comb, y_comb = shuffle(X_comb,y_comb)
Xsplits=[]
Ysplits=[]
size = X_comb.shape[0]
K=10
sizeOfsplit = size//K
for i in range(K):
    Xsplits.append(X_comb[i*sizeOfsplit:(i+1)*sizeOfsplit,:])
    Ysplits.append(y_comb[i*sizeOfsplit:(i+1)*sizeOfsplit])
splits = [ tuple(filter(lambda x: x!=i,tuple(range(K)))) for i in range(K)]
res = []
testRes=[]
for i in range(K):
    Xtrain = vstack(tuple(Xsplits[x] for x in splits[i]))
    ytrain = np.concatenate(tuple(Ysplits[x] for x in splits[i]))
    
    Xtest = Xsplits[i]
    ytest = np.copy(Ysplits[i])
    
    with tf.device('/CPU:0'):
        x_in = keras.layers.Input(shape=(xt.shape[1],))
        x = Dense(64, activation='relu')(x_in)
#         x = BatchNormalization()(x)
        Dropout(0.2)
        x = Dense(16, activation='relu')(x)  
        Dropout(0.2)
#         x = BatchNormalization()(x)
        y_out = Dense(1, activation='sigmoid')(x)
        model = keras.models.Model(x_in,y_out)
        model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
#         print(model.summary())
        training = model.fit(Xtrain,ytrain,batch_size=64,epochs=10, shuffle="batch", verbose=2, validation_split=0,validation_data=(Xtest, ytest))
        y_pred = model.predict(Xtest)
        y_cut = np.where(y_pred>0.5,1,0)
        macrof1 = f1_score(ytest, y_cut, zero_division=1)
        print("MACRO F1 : ",macrof1)

Train on 7704 samples, validate on 856 samples
Epoch 1/10
 - 8s - loss: 0.4031 - accuracy: 0.9181 - val_loss: 0.1479 - val_accuracy: 0.9650
Epoch 2/10
 - 8s - loss: 0.0529 - accuracy: 0.9904 - val_loss: 0.0827 - val_accuracy: 0.9731
Epoch 3/10
 - 8s - loss: 0.0109 - accuracy: 0.9990 - val_loss: 0.0741 - val_accuracy: 0.9743
Epoch 4/10
 - 8s - loss: 0.0039 - accuracy: 0.9999 - val_loss: 0.0725 - val_accuracy: 0.9743
Epoch 5/10
 - 8s - loss: 0.0020 - accuracy: 1.0000 - val_loss: 0.0727 - val_accuracy: 0.9755
Epoch 6/10
 - 8s - loss: 0.0012 - accuracy: 1.0000 - val_loss: 0.0741 - val_accuracy: 0.9755
Epoch 7/10
 - 8s - loss: 8.1116e-04 - accuracy: 1.0000 - val_loss: 0.0745 - val_accuracy: 0.9755
Epoch 8/10
 - 8s - loss: 5.6010e-04 - accuracy: 1.0000 - val_loss: 0.0750 - val_accuracy: 0.9755
Epoch 9/10
 - 8s - loss: 4.0072e-04 - accuracy: 1.0000 - val_loss: 0.0757 - val_accuracy: 0.9755
Epoch 10/10
 - 8s - loss: 2.9706e-04 - accuracy: 1.0000 - val_loss: 0.0768 - val_accuracy: 0.9766
MACRO 

Train on 7704 samples, validate on 856 samples
Epoch 1/10
 - 8s - loss: 0.4112 - accuracy: 0.8677 - val_loss: 0.1472 - val_accuracy: 0.9626
Epoch 2/10
 - 8s - loss: 0.0543 - accuracy: 0.9912 - val_loss: 0.0869 - val_accuracy: 0.9778
Epoch 3/10
 - 8s - loss: 0.0108 - accuracy: 0.9995 - val_loss: 0.0790 - val_accuracy: 0.9755
Epoch 4/10
 - 8s - loss: 0.0038 - accuracy: 1.0000 - val_loss: 0.0790 - val_accuracy: 0.9755
Epoch 5/10
 - 8s - loss: 0.0020 - accuracy: 1.0000 - val_loss: 0.0800 - val_accuracy: 0.9755
Epoch 6/10
 - 8s - loss: 0.0012 - accuracy: 1.0000 - val_loss: 0.0818 - val_accuracy: 0.9766
Epoch 7/10
 - 8s - loss: 7.5412e-04 - accuracy: 1.0000 - val_loss: 0.0836 - val_accuracy: 0.9766
Epoch 8/10
 - 8s - loss: 4.7851e-04 - accuracy: 1.0000 - val_loss: 0.0855 - val_accuracy: 0.9766
Epoch 9/10
 - 8s - loss: 3.1563e-04 - accuracy: 1.0000 - val_loss: 0.0876 - val_accuracy: 0.9755
Epoch 10/10
 - 8s - loss: 2.1861e-04 - accuracy: 1.0000 - val_loss: 0.0893 - val_accuracy: 0.9766
MACRO 

In [7]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from keras.layers import Embedding
from keras.layers import Bidirectional
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import BatchNormalization
from keras.layers import Concatenate
from keras.layers import Lambda
from keras.layers import LSTM
from keras.layers import Input
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers import Activation
from keras.layers import GRU

import keras.backend as kb
def attention_layer(inputs, neurons):
    x = keras.layers.Permute((2,1))(inputs)
    x = keras.layers.Dense(neurons, activation="softmax")(x)
    x = keras.layers.Permute((2,1), name="attention")(x)
    x = keras.layers.add([inputs, x])
    return x
def trainModelCNN(Xtrain,Xtrain_feat,embed,ytrain):
    with tf.device('/CPU:0'):
        x_in = keras.layers.Input(shape=(PADDING_SIZE,))
        x_feat = keras.layers.Input(shape=(SENT_LEVEL_FEATURES+UNIGRAMS_CNT,))
        unig = Dense(32, activation='relu')(x_feat)
        unig = Dense(16, activation='relu')(unig)
        
        x = Embedding(input_dim=embed.shape[0],output_dim=embed.shape[1],weights=[embed],input_length=PADDING_SIZE,trainable=False)(x_in)
        x1 = Conv1D(50,2, activation='relu',input_shape=(PADDING_SIZE,VECTOR_SIZE+EXTRA_VECTORS),padding='same') (x)
        x1 = MaxPooling1D(pool_size=2) (x1)
        x2 = Conv1D(50,3, activation='relu',input_shape=(PADDING_SIZE,VECTOR_SIZE+EXTRA_VECTORS),padding='same') (x)
        x2 = MaxPooling1D(pool_size=2) (x2)
        x3 = Conv1D(50,4, activation='relu',input_shape=(PADDING_SIZE,VECTOR_SIZE+EXTRA_VECTORS),padding='same') (x)
        x3 = MaxPooling1D(pool_size=2) (x3)
        x4 = Conv1D(50,5, activation='relu',input_shape=(PADDING_SIZE,VECTOR_SIZE+EXTRA_VECTORS),padding='same') (x)
        x4 = MaxPooling1D(pool_size=2) (x4)
        x = Concatenate(axis=2)([x1,x2,x3,x4])
        x = Flatten()(x)
#         x = attention_layer(x, neurons=PADDING_SIZE)
#         x = Bidirectional(keras.layers.LSTM(units=50, dropout=0.2, return_sequences=True))(x)
#         x = Bidirectional(keras.layers.LSTM(units=50, dropout=0.2))(x)
        
        x = Dense(32, activation='relu')(x)
        x = Dense(10, activation='relu')(x)
        
        x = Concatenate(axis=1)([x]+[unig])
        
        y_out = Dense(1, activation='sigmoid')(x)
        
        model = keras.models.Model([x_in,x_feat], y_out)
        model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
        print(model.summary())

        training = model.fit([Xtrain,Xtrain_feat],ytrain,batch_size=512,epochs=20, shuffle=True, verbose=1, validation_split=0)
        
        return training,model

In [21]:
test=pd.DataFrame(pd.read_csv("Constraint_English_Test - Sheet1.csv"))
train=pd.DataFrame(pd.read_csv("Constraint_English_Train - Sheet1.csv"))
val=pd.DataFrame(pd.read_csv("Constraint_English_Val - Sheet1.csv"))

processdf(test)
processdf(train)
processdf(val)

vecmodel = pickle.loads(open('word2Vec','rb').read())
Xlist,ylist,XFlist,embed = getEmbeddingsFeatures([train,val,test],vecmodel)

(6420, 40) (6420, 1)
(2140, 40) (2140, 1)
(2140, 40) (2140, 1)


In [26]:
X_comb = np.vstack(Xlist[0:2])
XF_comb = np.vstack(XFlist[0:2])
y_comb = np.vstack(ylist[0:2])

(8560, 1)

In [39]:
from sklearn.linear_model import PassiveAggressiveClassifier
Xsplits=[]
XFsplits=[]
Ysplits=[]
size = X_comb.shape[0]
K=10
sizeOfsplit = size//K
for i in range(K):
    Xsplits.append(X_comb[i*sizeOfsplit:(i+1)*sizeOfsplit,:])
    XFsplits.append(XF_comb[i*sizeOfsplit:(i+1)*sizeOfsplit,:])
    Ysplits.append(y_comb[i*sizeOfsplit:(i+1)*sizeOfsplit,:])
splits = [ tuple(filter(lambda x: x!=i,tuple(range(K)))) for i in range(K)]
res = []
for i in range(K):
    Xtrain = np.concatenate(tuple(Xsplits[x] for x in splits[i]))
    XFtrain = np.concatenate(tuple(XFsplits[x] for x in splits[i]))
    ytrain = np.concatenate(tuple(Ysplits[x] for x in splits[i]))
    
    Xtest = np.copy(Xsplits[i])
    XFtest = np.copy(XFsplits[i])
    ytest = np.copy(Ysplits[i])
    
    print(Xtrain.shape,XFtrain.shape,ytrain.shape)
    
    clf = PassiveAggressiveClassifier(max_iter=1000, random_state=0,tol=1e-3)
    clf.fit(Xtrain,ytrain)
    y_pred_pac=clf.predict(Xtest)
    scoref1 = f1_score(y_pred_pac,ytest,average="macro")
    print("Fold : "+str(i)+" F1 Score",scoref1)
    

In [50]:
Xsplits=[]
XFsplits=[]
Ysplits=[]
size = X_comb.shape[0]
K=5
sizeOfsplit = size//K
for i in range(K):
    Xsplits.append(X_comb[i*sizeOfsplit:(i+1)*sizeOfsplit,:])
    XFsplits.append(XF_comb[i*sizeOfsplit:(i+1)*sizeOfsplit,:])
    Ysplits.append(y_comb[i*sizeOfsplit:(i+1)*sizeOfsplit,:])
splits = [ tuple(filter(lambda x: x!=i,tuple(range(K)))) for i in range(K)]
res = []
for i in range(K):
    Xtrain = np.concatenate(tuple(Xsplits[x] for x in splits[i]))
    XFtrain = np.concatenate(tuple(XFsplits[x] for x in splits[i]))
    ytrain = np.concatenate(tuple(Ysplits[x] for x in splits[i]))
    
    Xtest = np.copy(Xsplits[i])
    XFtest = np.copy(XFsplits[i])
    ytest = np.copy(Ysplits[i])
    
    print(Xtrain.shape,XFtrain.shape,ytrain.shape)
    
    training,model = trainModelCNN(Xtrain,XFtrain,embed,ytrain)
    y_pred = model.predict([Xtest,XFtest])
    macrof1 = getF1(getConfusion(ytest,y_pred,2,cutoff=0.8))[0][2]
    res.append((model,macrof1))
    print("Fold : "+str(i)+" F1 Score",macrof1)

(6848, 40) (6848, 4075) (6848, 1)
Model: "model_12"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_27 (InputLayer)           (None, 40)           0                                            
__________________________________________________________________________________________________
embedding_14 (Embedding)        (None, 40, 203)      4982838     input_27[0][0]                   
__________________________________________________________________________________________________
conv1d_53 (Conv1D)              (None, 40, 50)       20350       embedding_14[0][0]               
__________________________________________________________________________________________________
conv1d_54 (Conv1D)              (None, 40, 50)       30500       embedding_14[0][0]               
_________________________________________________________

Epoch 1/20
6848/6848 [==============================] - 12s 2ms/step - loss: 1.3229 - accuracy: 0.5799
Epoch 2/20
6848/6848 [==============================] - 11s 2ms/step - loss: 0.5604 - accuracy: 0.7373
Epoch 3/20
2048/6848 [=======>......................] - ETA: 5s - loss: 0.4409 - accuracy: 0.8218

KeyboardInterrupt: 

In [16]:
from sklearn.metrics import f1_score

test=pd.DataFrame(pd.read_csv("Constraint_English_Test - Sheet1.csv"))
train=pd.DataFrame(pd.read_csv("Constraint_English_Train - Sheet1.csv"))
val=pd.DataFrame(pd.read_csv("Constraint_English_Val - Sheet1.csv"))

processdf(test)
processdf(train)
processdf(val)

# vecmodel = saveWord2VecModel([train,val,test])

vecmodel = pickle.loads(open('word2Vec','rb').read())
Xlist,ylist,XFlist,embed = getEmbeddingsFeatures([train,val,test],vecmodel)

training,model = trainModelCNN(Xlist[0],XFlist[0],embed,ylist[0])

y_pred = model.predict([Xlist[1],XFlist[1]])
yPred = np.where(y_pred > 0.5, 1, 0)
func_Eval(ylist[1].flatten(),yPred.flatten())
print("F1 Score",getF1(getConfusion(ylist[1],y_pred,2,cutoff=0.8))[0][2])
# for i in range(1,100):
#     cutoff = (1/100) * i
#     print("F1 Score",getF1(getConfusion(ylist[1],y_pred,2,cutoff=cutoff))[0][2])

NameError: name 'getLexicon1' is not defined

In [20]:
y_pred = model.predict([Xlist[2],XFlist[2]])
yPred = np.where(y_pred > 0.8, 1, 0)
f = open("answer.txt", "a")
f.write("id,label\n")
c=1
for i in range(len(yPred.flatten())):
    if(yPred[i]==0):
        f.write(str(i+1)+","+"fake")
    else:
        f.write(str(i+1)+","+"real")
    if(i!=len(yPred.flatten())-1):
        f.write("\n")
f.close()

In [228]:
test

,id,tweet,label,tweet_id
0,1,"[our, daily, update, is, published, state, rep...",0,"[36, 97, 73, 10, 170, 23, 49, 2, 72, 21, 2, 72..."
1,2,"[alfalfa, is, the, only, cure, for, covidz]",0,"[21455, 10, 3, 99, 158, 12, 8]"
2,3,"[president, trump, asked, what, he, would, do,...",0,"[139, 93, 765, 213, 108, 230, 114, 66, 108, 61..."
3,4,"[state, reported, X, death, we, are, still, se...",0,"[23, 49, 2, 27, 25, 14, 191, 1109, 6, 2204, 20..."
4,5,"[this, is, the, sixth, time, a, global, health...",0,"[29, 10, 3, 3169, 87, 6, 416, 57, 467, 22, 38,..."
...,...,...,...,...
2135,2136,"[coronavirusupdates, statewise, detail, of, to...",0,"[128, 1225, 161, 4, 40, 45, 8, 11, 545, 2, 215..."
2136,2137,"[tonight, X, midnight, onwards, disaster, mana...",0,"[2010, 2, 4094, 24623, 1248, 623, 506, 22, 38,..."
2137,2138,"[X, new, case, of, covidznigeria, plateau, X, ...",0,"[2, 18, 11, 4, 258, 295, 2, 361, 2, 224, 2, 14..."
2138,2139,"[rt, cdcemergency, dyk, cdcgovs, onestop, shop...",0,"[82, 1442, 1405, 24625, 6137, 1244, 12, 8, 983..."


# BERT Model

In [9]:
PADDING_SIZE = 30
from transformers import *
from transformers import BertTokenizer, TFBertModel, BertConfig
corpus = pd.concat([train,test,val])
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)
input_ids=[]
attention_masks=[]
segment_ids = []
decoded=[]
for i in tqdm(range(len(corpus))):
    bert_inp= bert_tokenizer.encode_plus(corpus.iloc[i]['tweet'],add_special_tokens=True,max_length =PADDING_SIZE,pad_to_max_length = True,return_attention_mask=True)
    input_ids.append(bert_inp['input_ids'])
    attention_masks.append(bert_inp['attention_mask'])
    decoded.append(bert_tokenizer.decode(bert_inp['input_ids']))
    segment_ids.append([0]*PADDING_SIZE)
corpus['input_ids'] = input_ids
corpus['segment_ids'] = segment_ids
corpus['attention_mask'] = attention_masks
corpus['decoded'] = decoded
corpus

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertForSequenceClassification: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['dropout_37', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|                                                                 

,id,tweet,label,input_ids,segment_ids,attention_mask,decoded
0,1,the cdc currently reports X deaths in general ...,1.0,"[101, 1996, 26629, 2747, 4311, 1060, 6677, 199...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[CLS] the cdc currently reports x deaths in ge...
1,2,states reported X deaths a small rise from las...,1.0,"[101, 2163, 2988, 1060, 6677, 1037, 2235, 4125...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[CLS] states reported x deaths a small rise fr...
2,3,politically correct woman almost uses pandemic...,0.0,"[101, 10317, 6149, 2450, 2471, 3594, 6090, 320...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[CLS] politically correct woman almost uses pa...
3,4,indiafightscorona we have X covid testing labo...,1.0,"[101, 2634, 20450, 9363, 26788, 2057, 2031, 10...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[CLS] indiafightscorona we have x covid testin...
4,5,populous states can generate large case counts...,1.0,"[101, 20151, 2163, 2064, 9699, 2312, 2553, 929...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[CLS] populous states can generate large case ...
...,...,...,...,...,...,...,...
2135,2136,donald trump wrongly claimed that new zealand ...,0.0,"[101, 6221, 8398, 29116, 3555, 2008, 2047, 341...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[CLS] donald trump wrongly claimed that new ze...
2136,2137,current understanding is covidz spreads mostly...,1.0,"[101, 2783, 4824, 2003, 2522, 17258, 2480, 208...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[CLS] current understanding is covidz spreads ...
2137,2138,nothing screams i am sat around doing fuck all...,0.0,"[101, 2498, 11652, 1045, 2572, 2938, 2105, 272...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[CLS] nothing screams i am sat around doing fu...
2138,2139,birx says covidz outbreak not under control be...,0.0,"[101, 12170, 2099, 2595, 2758, 2522, 17258, 24...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[CLS] birx says covidz outbreak not under cont...


In [10]:
train1 = corpus.iloc[:6420]
test1 = corpus.iloc[:8560]
val1 = corpus.iloc[8560:]

Xtrain_id = np.vstack(train1['input_ids'])
Xtrain_att = np.vstack(train1['attention_mask'])
Xtrain_seg = np.vstack(train1['segment_ids'])

Xtest_id = np.vstack(test1['input_ids'])
Xtest_att = np.vstack(test1['attention_mask'])
Xtest_seg = np.vstack(test1['segment_ids'])

Xval_id = np.vstack(test1['input_ids'])
Xval_att = np.vstack(test1['attention_mask'])
Xval_seg = np.vstack(test1['segment_ids'])

trainComb = (Xtrain_id,Xtrain_att,Xtrain_seg)
valComb = (Xval_id,Xval_att,Xval_seg)

ytrain = np.array(train1['label'])
yval = np.array(val1['label'])

print(Xtrain_id.shape,Xtest_id.shape,Xval_id.shape)
print(Xtrain_att.shape,Xtest_att.shape,Xval_att.shape)
print(Xtrain_seg.shape,Xtest_seg.shape,Xval_seg.shape)

print(ytrain.shape,yval.shape)

(6420, 30) (8560, 30) (8560, 30)
(6420, 30) (8560, 30) (8560, 30)
(6420, 30) (8560, 30) (8560, 30)
(6420,) (2140,)


In [13]:
import tensorflow_hub as hub
module_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2'
bert_layer = hub.KerasLayer(module_url, trainable=True)

#MODEL 2
input_word_ids = tf.keras.Input(shape=(PADDING_SIZE,), dtype=tf.int32, name="input_word_ids")
input_mask = tf.keras.Input(shape=(PADDING_SIZE,), dtype=tf.int32, name="input_mask")
segment_ids = tf.keras.Input(shape=(PADDING_SIZE,), dtype=tf.int32, name="segment_ids")

pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
clf_output = sequence_output[:, 0, :]
net = tf.keras.layers.Dense(64, activation='relu')(clf_output)
net = tf.keras.layers.Dropout(0.2)(net)
# net = tf.keras.layers.Dense(32, activation='relu')(net)
# net = tf.keras.layers.Dropout(0.2)(net)
out = tf.keras.layers.Dense(1, activation='softmax')(net)

model = tf.keras.models.Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
model.compile(tf.keras.optimizers.Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())

#FIT
checkpoint = tf.keras.callbacks.ModelCheckpoint('model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, verbose=1)
train_history = model.fit(trainComb,ytrain, validation_split=0.2,epochs=1,callbacks=[checkpoint, earlystopping],batch_size=256,verbose=1)

In [12]:
import tensorflow as tf
from tensorflow import keras
model = keras.Sequential()
model.add(keras.layers.Dense(10, input_shape=(30,)))
model.add(keras.layers.Dense(1, input_shape=(30,)))
model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['binary_accuracy'])
model.fit(x=Xtrain_id,y=ytrain,shuffle=True,epochs=5,batch_size=2000)

Epoch 1/5
4/4 [==============================] - 0s 2ms/step - loss: 7.7423 - binary_accuracy: 0.4961
Epoch 2/5
4/4 [==============================] - 0s 3ms/step - loss: 7.7423 - binary_accuracy: 0.4961
Epoch 3/5
4/4 [==============================] - 0s 4ms/step - loss: 7.7423 - binary_accuracy: 0.4961
Epoch 4/5
4/4 [==============================] - 0s 4ms/step - loss: 7.7423 - binary_accuracy: 0.4961
Epoch 5/5
4/4 [==============================] - 0s 3ms/step - loss: 7.7423 - binary_accuracy: 0.4961
